In [1]:
using Printf
using CPUTime
include("model_chessgame.jl")
include("spu_chessgame.jl")
using Gurobi

function createFolder(directory)
    try
        if isdir(directory) == false
            mkdir(directory)
        end
    catch
        println(string("Error:Creating directory ", directory))
    end
end

createFolder (generic function with 1 method)

# Draw randomly the parameters of the influence diagram

In [32]:
#Time horizon
T=8

#S space
S = 1:6

#O space
O = 1:4

#U space
U = 1:3

#A space
A = 1:4

#V space
V = 1:5

#p_init(s) = P(S_1=s)
p_init_s = rand(length(S))
p_init_s = p_init_s ./ sum(p_init_s)

#p_trans(s,v,s') = P(S_{t+1} = s' |  S_t = s, V_t = v)
p_trans_s = rand(length(S),length(V),length(S))
p_trans_s = p_trans_s ./ sum(p_trans_s,dims=3)

# p_trans_v(o,a,v) = P(V_t = v | O_t =o, A_t =a)
p_trans_v = rand(length(O), length(A), length(V))
p_trans_v = p_trans_v ./ sum(p_trans_v,dims=3)

# p_emis_o(s,o) = P(O_t = o | S_t =s)
p_emis_o = rand(length(S),length(O))
p_emis_o = p_emis_o ./ sum(p_emis_o,dims=2)

# p_emis_u(o,u) = P(U_t = u | O_t =o)
p_emis_u = rand(length(O),length(U))
p_emis_u = p_emis_u ./ sum(p_emis_u,dims=2)

reward = 10*rand(length(V))
#############################################################################

5-element Array{Float64,1}:
 2.4408482341443816
 1.4776864551359559
 7.356373372231788 
 7.393507961447609 
 1.8699808972905863

# Preprocessing of cut probabilities

In [33]:
## Compute conditional probabilities p_{o | su}
p_cuts = zeros(length(S),length(U),length(O))
for s in S, u in U, o in O
	if sum(p_emis_o[s,oo]*p_emis_u[oo,u] for oo in O) == 0
		p_cuts[s,u,o] = 0
	else 
		p_cuts[s,u,o] = (p_emis_o[s,o]*p_emis_u[o,u])/(sum(p_emis_o[s,oo]*p_emis_u[oo,u] for oo in O))
	end
end

# Run SPU algorithm

In [34]:
##################################SPU algorithm########
@printf "================================ SPU algorithm =========================================\n"
@time SPU_Time = SPU_obj, delta, iterations = SPU_algorithm(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward)
SPU_delta,μ_svs,μ_sv,μ_soav,μ_soa,μ_sou,μ_soua,μ_s = marginals_warmstart(T,S,O,U,A,V,delta,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward)
@printf "=================================== Done ======================================\n"

================================ SPU algorithm =========================================
=============Cycle 0==================
=============Cycle 1==================
  0.032891 seconds (355.68 k allocations: 6.748 MiB)
=================================== Done ======================================


# Solve MILP on Polytope $\overline{\mathcal{Q}}^1$

In [35]:
###################Polytope Q^1#########################
@printf "================================ Polytope Q^1 =========================================\n"
#Define the bounds b in McCormick inequalities
#b=1				
bounds = ones(T,length(S), length(O), length(U))

#Solve the linear relaxation
m = model(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_p1_relax = objective_value(m_lp)

#Solve the MILP
m = model(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward,bounds)
m = set_initial_solution(m,T,S,O,U,A,V,SPU_delta,μ_svs,μ_sv,μ_soav,μ_soa,μ_sou,μ_soua,μ_s)
optimize!(m)

#Set the results_
milp_time_p1 = solve_time(m)
milp_obj_p1 = objective_value(m)
milp_bestbound_p1 = objective_bound(m)
milp_gap_p1 = ((milp_bestbound_p1 - milp_obj_p1)/milp_bestbound_p1)*100
gap_p1_with_spu = ((milp_obj_p1 - SPU_obj)/SPU_obj)*100

================================ Polytope Q^1 =========================================
Academic license - for non-commercial use only
Academic license - for non-commercial use only


-1.845476452014029e-14

# Solve MILP on Polytope $\overline{\mathcal{Q}}^b$

In [36]:
###################Polytope Q^b#########################
@printf "================================ Polytope Q^b =========================================\n"
#Define the bounds b in McCormick inequalities
#Compute the bounds using propagation
bounds = compute_bounds(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward)

#Solve the linear relaxation
m = model(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_pb_relax = objective_value(m_lp)

#Solve the MILP
m = model(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward,bounds)
m = set_initial_solution(m,T,S,O,U,A,V,delta,μ_svs,μ_sv,μ_soav,μ_soa,μ_sou,μ_soua,μ_s)
optimize!(m)


#Set the results
milp_time_pb = solve_time(m)
milp_obj_pb = objective_value(m)
milp_bestbound_pb = objective_bound(m)
milp_gap_pb = ((milp_bestbound_pb - milp_obj_pb)/milp_bestbound_pb)*100
gap_pb_with_spu = ((milp_obj_pb - SPU_obj)/SPU_obj)*100

================================ Polytope Q^b =========================================
Academic license - for non-commercial use only
Academic license - for non-commercial use only


-1.845476452014029e-14

# Solve MILP on Polytope $\mathcal{Q}^{\perp \!\!\! \perp,1}$

In [37]:
#######################Polytope Q^{free,1}##############################################
@printf "================================ Polytope Q^{free,1} =========================================\n"
#Define the bounds b in McCormick inequalities
#b=1
bounds = ones(T,length(S), length(O), length(U))

#Solve the linear relaxation
m = model_cuts(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,p_cuts,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_pfree_relax = objective_value(m_lp)

#Solve the MILP
m = model_cuts(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,p_cuts,reward,bounds)
m = set_initial_solution(m,T,S,O,U,A,V,delta,μ_svs,μ_sv,μ_soav,μ_soa,μ_sou,μ_soua,μ_s)
optimize!(m)

#Set the results_
milp_time_pfree = solve_time(m)
milp_obj_pfree = objective_value(m)
milp_bestbound_pfree = objective_bound(m)
milp_gap_pfree = ((milp_bestbound_pfree-milp_obj_pfree)/milp_bestbound_pfree)*100
gap_pfree_with_spu = ((milp_obj_pfree - SPU_obj)/SPU_obj)*100

================================ Polytope Q^{free,1} =========================================
Academic license - for non-commercial use only
Academic license - for non-commercial use only


-1.845476452014029e-14

# Solve MILP on Polytope $\mathcal{Q}^{\perp \!\!\! \perp,b}$

In [38]:
#######################Polytope Q^free \cap Q^b##############################################
@printf "================================ Polytope Q^{free,b} =========================================\n"
#Define the bounds b in McCormick inequalities
#Compute the bounds using propagation
bounds = compute_bounds(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,reward)

#Solve the linear relaxation
m = model_cuts(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,p_cuts,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_pfree_pb_relax = objective_value(m_lp)

#Solve the MILP
m = model_cuts(T,S,O,U,A,V,p_init_s,p_trans_s,p_trans_v,p_emis_o,p_emis_u,p_cuts,reward,bounds)
m = set_initial_solution(m,T,S,O,U,A,V,delta,μ_svs,μ_sv,μ_soav,μ_soa,μ_sou,μ_soua,μ_s)
optimize!(m)

#Set the results
milp_time_pfree_pb = solve_time(m)
milp_obj_pfree_pb = objective_value(m)
milp_bestbound_pfree_pb = objective_bound(m)
milp_gap_pfree_pb = ((milp_bestbound_pfree_pb-milp_obj_pfree_pb)/milp_bestbound_pfree_pb)*100
gap_pfree_pb_with_spu = ((milp_obj_pfree_pb - SPU_obj)/SPU_obj)*100

================================ Polytope Q^{free,b} =========================================
Academic license - for non-commercial use only
Academic license - for non-commercial use only


-1.845476452014029e-14

In [39]:
#########################################Results##############################
@printf "============================================================================\n"
@printf "===================================================================\n"
@printf "===========================================================\n"
@printf "==============================================\n"
@printf "=============Results %.0f %.0f %.0f %.0f==================\n" T length(S) length(O) length(A)

@printf "Q^1 Integrality gap : %f \n" (1-milp_obj_p1/milp_obj_p1_relax)*100
@printf "Q^1 Time : %f \n" milp_time_p1
@printf "Q^1 Final gap value: %f \n" milp_gap_p1
@printf "Q^1 Gap with SPU: %f \n \n" gap_p1_with_spu

@printf "Q^b Integrality gap : %f \n" (1- milp_obj_pb/milp_obj_pb_relax)*100
@printf "Q^b Time : %f \n" milp_time_pb
@printf "Q^b Final gap value: %f \n" milp_gap_pb
@printf "Q^b Gap with SPU: %f \n \n" gap_pb_with_spu


@printf "Q^free Integrality gap : %f \n" (1- milp_obj_pfree/milp_obj_pfree_relax)*100
@printf "Q^free Time : %f \n" milp_time_pfree
@printf "Q^free Final gap value: %f \n" milp_gap_pfree
@printf "Q^free Gap with SPU: %f \n \n" gap_pfree_with_spu


@printf "Q^{free,b} Integrality gap : %f \n" (1- milp_obj_pfree_pb/milp_obj_pfree_pb_relax)*100
@printf "Q^{free,b} Time : %f \n" milp_time_pfree_pb
@printf "Q^{free,b} Final gap value: %f \n" milp_gap_pfree_pb
@printf "Q^{free,b} Gap with SPU: %f \n \n" gap_pfree_pb_with_spu

=============Results 8 6 4 4==================
Q^1 Integrality gap : 1.848471 
Q^1 Time : 1.590988 
Q^1 Final gap value: 0.000000 
Q^1 Gap with SPU: -0.000000 
 
Q^b Integrality gap : 1.175010 
Q^b Time : 2.279793 
Q^b Final gap value: 0.000000 
Q^b Gap with SPU: -0.000000 
 
Q^free Integrality gap : 0.000000 
Q^free Time : 0.137140 
Q^free Final gap value: 0.000000 
Q^free Gap with SPU: -0.000000 
 
Q^{free,b} Integrality gap : 0.000000 
Q^{free,b} Time : 0.273382 
Q^{free,b} Final gap value: 0.000000 
Q^{free,b} Gap with SPU: -0.000000 
 
